In [1]:
import phoebe
import numpy as np
import matplotlib.pyplot as plt
import emcee
import sys
%matplotlib inline

In [2]:
b = phoebe.default_binary(contact_binary = True)

In [3]:
#b.add_constraint('semidetached', 'primary')

In [4]:
#b.add_constraint('semidetached', 'secondary')

In [5]:
b['period@orbit'] = 1
b['sma@orbit'] = 1
#b['q@binary'] = 0.44
# b['incl@orbit'] = 83.5
# b['requiv@primary'] = 1.2
# b['requiv@secondary'] = 0.8
#b['teff@primary'] = 5660.
# b['teff@secondary'] = 5500.

In [6]:
#lc = np.loadtxt('data.lc')
v737 = np.loadtxt('KIC 4937217.txt')
vdata = v737[:,1]
flux = vdata/(-2.5)
flux10 = 10**flux
lc = v737
lc[:,1] = flux10
sigmas = np.diff(flux10,2).std()*np.ones(len(lc))

In [7]:
#b.add_dataset('lc', times=lc[:,0], fluxes=lc[:,1], sigmas=0.05*np.ones(len(lc)), passband='Johnson:B')
b.add_dataset('lc', times=lc[:,0], fluxes=lc[:,1], sigmas=sigmas, passband='Kepler:mean')

<ParameterSet: 78 parameters | contexts: figure, compute, constraint, dataset>

In [8]:
phoebe.interactive_checks_off()
phoebe.interactive_constraints_off()
b.set_value_all('irrad_method', 'none')

In [9]:
b.flip_constraint('compute_phases', 'compute_times')
b['compute_phases@lc@dataset'] = np.linspace(-0.5,0.5,len(lc))

In [10]:
def lnprob(x, adjpars, priors):
    #Check to see that all values are within the allowed limits:
    if not np.all([priors[i][0] < x[i] < priors[i][1] for i in range(len(priors))]):
         return -np.inf

    for i in range(len(adjpars)):
        b[adjpars[i]] = x[i]
    
    # Let's assume that our priors are uniform on the range of the physical parameter combinations.
    # This is already handled in Phoebe, which will throw an error if the system is not physical,
    # therefore it's easy to implement the lnprior as =0 when system checks pass and =-inf if they don't.
    # Here we'll 'package' this in a simple try/except statement:
    
    try:
        b.run_compute(irrad_method='none')

        # sum of squares of the residuals
        fluxes_model = b['fluxes@model'].interp_value(times=lc[:,0])
        #lnp = -0.5*np.sum((fluxes_model-b['value@fluxes@dataset'])**2 / b['value@sigmas@dataset']**2)
        mag_model = -2.5*np.log10(fluxes_model) 
        mag_model = mag_model-np.mean(mag_model)
        mag_value = -2.5*np.log10(b['value@fluxes@dataset'])
        mag_value = mag_value-np.mean(mag_value)
        lnp = -0.5*np.sum((mag_model-mag_value)**2 / b['value@sigmas@dataset']**2)
        
        #print('inp is ok!')

    except:
        lnp = -np.inf

    sys.stderr.write("lnp = %e\n" % (lnp))

    return lnp

In [11]:
from scipy.stats import norm
def run(adjpars, priors, nwalkers, niter):
    ndim = len(adjpars)

   

    p0 = np.array([[p[0] + (p[1]-p[0])*np.random.rand() for p in priors] for i in range(nwalkers)])

#     pool = MPIPool()
#     if not pool.is_master():
#         pool.wait()
#         sys.exit(0)

    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=[adjpars, priors])

    pos, prob, state = sampler.run_mcmc(p0, niter)
    
    print(p0)
    
    for i in range(ndim):
        plt.figure()
        y = sampler.flatchain[:,i]
        n, bins, patches = plt.hist(y, 200, density=1, color="b", alpha=0.45)
        plt.title("Dimension {0:d}".format(i))
        
        mu = np.average(y)
        sigma = np.std(y)       
        print ("mu,", "sigma = ", mu, sigma)

        bf = norm.pdf(bins, mu, sigma)
        l = plt.plot(bins, bf, 'k--', linewidth=2.0)
         
    #plt.show()
    
    return pos

#     pool.close()



In [ ]:
adjpars = ['requiv@primary', 'incl@orbit', 'q@binary', 'teff@primary', 'teff@secondary']
priors = [(0.45,0.53), (64.0, 68.0), (0.2, 0.4), (6000, 6200), (5800, 6000.)]
nwalkers = 100

niters = 20
state = None

import time

time1 = time.time()
postion = run(adjpars, priors, nwalkers, niters)
time2 = time.time()

run_time = time2-time1

print(run_time)

lnp = -inf
lnp = -inf
lnp = -inf
100%|██████████| 325/325 [00:11<00:00, 28.79it/s]
lnp = -6.114149e+03
lnp = -inf
lnp = -inf
100%|██████████| 325/325 [00:10<00:00, 29.58it/s]
lnp = -8.526477e+03
lnp = -inf
100%|██████████| 325/325 [00:10<00:00, 29.55it/s]
lnp = -7.623094e+04
100%|██████████| 325/325 [00:10<00:00, 30.00it/s]
lnp = -3.201248e+03
lnp = -inf
100%|██████████| 325/325 [00:10<00:00, 29.89it/s]
lnp = -3.147307e+04
lnp = -inf
100%|██████████| 325/325 [00:10<00:00, 30.29it/s]
lnp = -6.230134e+03
lnp = -inf
lnp = -inf
lnp = -inf
lnp = -inf
lnp = -inf
lnp = -inf
lnp = -inf
100%|██████████| 325/325 [00:11<00:00, 28.61it/s]
lnp = -3.802583e+04
100%|██████████| 325/325 [00:11<00:00, 28.96it/s]
lnp = -1.070144e+04
lnp = -inf
100%|██████████| 325/325 [00:11<00:00, 29.47it/s]
lnp = -7.541190e+03
100%|██████████| 325/325 [00:11<00:00, 29.33it/s]
lnp = -9.781702e+03
lnp = -inf
100%|██████████| 325/325 [00:11<00:00, 28.11it/s]
lnp = -1.356882e+04
100%|██████████| 325/325 [00:11<00:00, 29.4

100%|██████████| 325/325 [00:11<00:00, 28.79it/s]
lnp = -2.652301e+04
100%|██████████| 325/325 [00:12<00:00, 27.06it/s]
lnp = -4.810160e+03
lnp = -inf
lnp = -inf
100%|██████████| 325/325 [00:11<00:00, 27.48it/s]
lnp = -5.163693e+03
100%|██████████| 325/325 [00:11<00:00, 27.47it/s]
lnp = -2.759898e+04
lnp = -inf
lnp = -inf
100%|██████████| 325/325 [00:11<00:00, 28.45it/s]
lnp = -8.739195e+03
lnp = -inf
100%|██████████| 325/325 [00:11<00:00, 27.73it/s]
lnp = -8.368021e+03
lnp = -inf
100%|██████████| 325/325 [00:11<00:00, 27.67it/s]
lnp = -5.334691e+04
100%|██████████| 325/325 [00:11<00:00, 29.19it/s]
lnp = -2.290906e+04
100%|██████████| 325/325 [00:11<00:00, 27.93it/s]
lnp = -6.027405e+04
100%|██████████| 325/325 [00:11<00:00, 27.85it/s]
lnp = -1.056689e+04
100%|██████████| 325/325 [00:11<00:00, 28.49it/s]
lnp = -3.299584e+04
lnp = -inf
lnp = -inf
lnp = -inf
100%|██████████| 325/325 [00:11<00:00, 28.14it/s]
lnp = -1.676646e+04
lnp = -inf
100%|██████████| 325/325 [00:11<00:00, 29.06it/s]


In [ ]:

from matplotlib.pyplot import cm 

mod = b
position = postion
times = lc[:,0]
color=cm.rainbow(np.linspace(0,1,nwalkers))

for i,c in zip(range(nwalkers),color):
    
    mod['requiv@primary'] = position[-1-i,0]
    mod['incl@binary@orbit@component'] = position[-1-i,1]
    mod['q@binary'] = position[-1-i,2]
    mod['teff@primary'] = position[-1-i,3]
    mod['teff@secondary'] = position[-1-i,4]
    try:
        mod.run_compute(model='run{}'.format(i))
    except:
        print('it is error')


for i,c in zip(range(nwalkers),color):
    try:
        model = mod['fluxes@run{}'.format(i)].interp_value(times=times)

    #plt.figure(1)
        plt.plot(times,model,c=c)
    except:
        print('it is error')
    
plt.plot(times,lc[:,1],"k.")
plt.xlabel("phases")
plt.ylabel("Flux")


In [ ]:
mod['incl@binary@orbit@component']

In [ ]:
mod['q@binary']

In [ ]:
mod['teff@primary']

In [ ]:
mod['teff@secondary']

In [ ]:
from matplotlib.pyplot import cm 

mod = b
position = postion
times = lc[:,0]
color=cm.rainbow(np.linspace(0,1,nwalkers))

for i,c in zip(range(nwalkers),color):
    
    mod['requiv@primary'] = position[-1-i,0]
    mod['incl@binary@orbit@component'] = position[-1-i,1]
    mod['q@binary'] = position[-1-i,2]
    mod['teff@primary'] = position[-1-i,3]
    mod['teff@secondary'] = position[-1-i,4]
    try:
        mod.run_compute(model='run{}'.format(i))
    except:
        print('it is error')


for i,c in zip(range(nwalkers),color):
    try:
        model = mod['fluxes@run{}'.format(i)].interp_value(times=times)

    #plt.figure(1)
        rmodel = -2.5*np.log10(model)
        resultmodel = rmodel-np.mean(rmodel)
        plt.plot(times,resultmodel,c=c)
    except:
        print('it is error')

lmodel = -2.5*np.log10(lc[:,1])  
lmag = lmodel-np.mean(lmodel)
ax = plt.gca()
plt.plot(times,lmag,"k.")
plt.xlabel("phases")
plt.ylabel("Mag")
ax.yaxis.set_ticks_position('left')
ax.invert_yaxis()